In [2]:
#Import de pandas
import pandas as pd

In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel

In [4]:
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     Hostname mismatch, certificate is not valid for
[nltk_data]     'raw.githubusercontent.com'. (_ssl.c:1002)>
[nltk_data] Error loading omw-1.4: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     Hostname mismatch, certificate is not valid for
[nltk_data]     'raw.githubusercontent.com'. (_ssl.c:1002)>


False

In [5]:
#Chargement des données
data = pd.read_csv("../../Donnees/analyse_texte/csv/cancer_sport_analyse.csv")
data2 = pd.read_csv("../../Donnees/analyse_texte/csv/cancer_fasting_analyse.csv")
data3 = pd.read_csv("../../Donnees/analyse_texte/csv/cancer_cannabis.csv")
tweets_df = pd.concat([data,data2,data3])

In [6]:
tweets_df.head()

,Fichier,ID,Nb retweet,Nb like,Nb réponses,Nb citations,Hashtags,Texte,Mots,Phrases,Tags POS,Sentiment,Polarité,Subjectivité,Phrases nominales,Texte corrigé,Texte tokénizé
0,721916579533950976.json,721916579533950976,0,0,0,0.0,"['#CANCER', '#SPORT', '#PARENTING', '#SCHOOL',...",Is artificial turf giving athletes #cancer ? -...,"['Is', 'artificial', 'turf', 'giving', 'athlet...","[Sentence(""Is artificial turf giving athletes ...","[('Is', 'VBZ'), ('artificial', 'JJ'), ('turf',...","Sentiment(polarity=-0.6, subjectivity=1.0)",-0.6,1.0000,"['artificial turf', 'athletes # cancer', 'spor...",Is artificial turf giving athletes #cancer ? -...,"['Is', 'artificial', 'turf', 'giving', 'athlet..."
1,238304774825603072.json,238304774825603072,0,1,0,0.0,"['#CANCER', '#SPORT']",Frankel's victory at York made me feel '20 yea...,"['Frankel', ""'s"", 'victory', 'at', 'York', 'ma...","[Sentence(""Frankel's victory at York made me f...","[('Frankel', 'NNP'), (""'s"", 'POS'), ('victory'...","Sentiment(polarity=0.5, subjectivity=0.5)",0.5,0.5000,"['frankel', ""'s victory"", 'york', ""'20 years"",...",Fraenkel's victory at Work made me feel '20 ye...,"['Frankel', ""'s"", 'victory', 'at', 'York', 'ma..."
2,1007390700726575104.json,1007390700726575104,0,4,1,0.0,"['#SPORT', '#CANCER']",READ THIS! What an absolutely brilliant piece ...,"['READ', 'THIS', 'What', 'an', 'absolutely', '...","[Sentence(""READ THIS!""), Sentence(""What an abs...","[('READ', 'VB'), ('THIS', 'NN'), ('What', 'WP'...","Sentiment(polarity=0.5, subjectivity=0.5375)",0.5,0.5375,"['read this', 'brilliant piece', '# sport trib...",READ THIS! That an absolutely brilliant piece ...,"['READ', 'THIS', '!', 'What', 'an', 'absolutel..."
3,668965073889697795.json,668965073889697795,0,0,0,0.0,"['#CANCER', '#STROKE', '#SPORT']","MJA InSight News: #Cancer survivors, exercise ...","['MJA', 'InSight', 'News', 'Cancer', 'survivor...","[Sentence(""MJA InSight News: #Cancer survivors...","[('MJA', 'NNP'), ('InSight', 'NNP'), ('News', ...","Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0000,"['mja insight', '# cancer survivors', 'atopic ...","MJA night News: #Cancer survivor, exercise bar...","['MJA', 'InSight', 'News', ':', '#', 'Cancer',..."
4,1098650711049555968.json,1098650711049555968,2,1,0,0.0,"['#HEALTH', '#CANCER', '#SPORT']",When #health #cancer &amp; #sport are inextric...,"['When', 'health', 'cancer', 'amp', 'sport', '...","[Sentence(""When #health #cancer &amp; #sport a...","[('When', 'WRB'), ('health', 'NN'), ('cancer',...","Sentiment(polarity=0.6000000000000001, subject...",0.6,0.9000,"['# health # cancer', '# sport', 'amazing', 'd...",When #health #cancer &amp; #sport are inextric...,"['When', '#', 'health', '#', 'cancer', '&', 'a..."


In [7]:
#La racinisation (stemming) consiste à transformer les mots en forme radicale c’est ce que fait la fonction SnowballStemmer de Nltk.
stemmer = SnowballStemmer('english') #ici anglais car nos tweets sont en anglais

In [8]:
#Toutes les lignes de la colonne "Texte corrigé" n'étaient pas toutes de types 'str'
tweets_df['Texte corrigé'] = tweets_df['Texte corrigé'].astype(str)

#Suppression des liens commençant par http et https  
def remove_url(text):
 return re.sub(r'https?:\S*', '', text)
tweets_df['Texte corrigé']=tweets_df['Texte corrigé'].apply(remove_url)
def remove_url1(text):
 return re.sub(r'http?:\S*', '', text)
tweets_df['Texte corrigé']=tweets_df['Texte corrigé'].apply(remove_url1)

In [9]:
#opération de mettre les mots sous leur forme canonique 
#ex: running devient run 
def lemmatize_stemming(text) :
  return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='n'))

In [10]:
#suppression des stopwords (the, or, and…) et des mots de moins de 3 lettres
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


# Imprimez le résultat intermédiaire pour chaque document
processed_docs = [preprocess(doc) for doc in tweets_df]
#for doc in processed_docs:
    # Concaténez les mots pour former une phrase
    #doc_text = ' '.join(doc)
    #print(preprocess(doc_text))
    



In [13]:
# On stocke les données dans un dictionnaire Gensim et on les convertit au format Bag Of Words(bow).
#On a alors un couple mot/nb d'occurence

# Utilisez la colonne "Texte corrigé" pour le prétraitement
processed_docs = [preprocess(doc) for doc in tweets_df['Texte corrigé']]

# Créez le dictionnaire et le corpus
dictionary = gensim.corpora.Dictionary(processed_docs)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Créez et entraînez le modèle LDA
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=15, # Le nombre de topics à extraire
                                       id2word=dictionary, # Le mapping entre les identifiants et les mots
                                       passes=3000) # Le nombre d'itérations dans le corpus/ passages




# Affichez les topics
topics = []
for idx, topic in lda_model.print_topics(-1) :
    print("Topic: {} -> Words: {}".format(idx, topic))
    topics.append(topic)

In [12]:
#Mesure de la cohérence du modèle
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary)
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.22367904920391116


In [17]:
#On garde les résultats dans un df
all_topic_model = []
for i in range(len(topics)):
  str = topics[i].split(' + ')
  topic_model = []
  for j in range(10):
    weight = str[j][0:5]
    word = str[j][7:len(str[j])-1]
    topic_model.append((weight, word))
  all_topic_model.append(topic_model)

In [18]:
df_topic_model = pd.DataFrame(all_topic_model)
df_topic_model.rename(index = {0: "Topic 1", 1: "Topic 2", 2: "Topic 3", 3: "Topic 4", 4: "Topic 5", 5: "Topic 6", 6: "Topic 7", 7: "Topic 8", 8: "Topic 9", 9: "Topic 10"})

,0,1,2,3,4,5,6,7,8,9
Topic 1,"(0.083, cancer)","(0.050, sport)","(0.031, port)","(0.025, chariti)","(0.021, research)","(0.017, music)","(0.016, cr_uk)","(0.016, ruin)","(0.016, baffl)","(0.016, nightout)"
Topic 2,"(0.084, fast)","(0.069, cancer)","(0.032, veto)","(0.029, fight)","(0.024, natur)","(0.022, nutrit)","(0.018, way)","(0.016, essentialoil)","(0.016, homeopath)","(0.015, benefit)"
Topic 3,"(0.060, cancer)","(0.045, fast)","(0.019, worldwid)","(0.019, healthyfood)","(0.018, naturaleh)","(0.018, ship)","(0.017, mang)","(0.017, caribbean)","(0.016, sport)","(0.015, patrickdelv)"
Topic 4,"(0.092, cancer)","(0.059, fast)","(0.043, health)","(0.043, wast)","(0.017, diet)","(0.014, nutrit)","(0.014, diabet)","(0.013, immun)","(0.010, benefit)","(0.010, diseas)"
Topic 5,"(0.067, cancer)","(0.029, sport)","(0.017, wast)","(0.013, fast)","(0.012, health)","(0.009, diet)","(0.008, diseas)","(0.008, nutrit)","(0.007, research)","(0.006, month)"
Topic 6,"(0.055, cancer)","(0.025, sport)","(0.014, port)","(0.009, fight)","(0.006, footbal)","(0.006, youtub)","(0.005, exercis)","(0.005, carcinoma)","(0.005, today)","(0.005, sleep)"
Topic 7,"(0.062, cancer)","(0.055, sport)","(0.016, chariti)","(0.015, news)","(0.014, awar)","(0.013, fundrais)","(0.013, event)","(0.012, support)","(0.012, pink)","(0.011, novemb)"
Topic 8,"(0.057, cancer)","(0.025, fast)","(0.018, sport)","(0.007, heal)","(0.007, port)","(0.006, talk)","(0.006, genetherapi)","(0.006, dy)","(0.005, brain)","(0.005, bitch)"
Topic 9,"(0.103, cancer)","(0.058, fast)","(0.024, wast)","(0.017, help)","(0.014, chemotherapi)","(0.011, reduc)","(0.010, breast)","(0.010, health)","(0.010, effect)","(0.010, studi)"
Topic 10,"(0.074, cancer)","(0.033, wast)","(0.030, fast)","(0.024, health)","(0.021, scientist)","(0.020, diet)","(0.019, trigger)","(0.018, right)","(0.016, stem)","(0.015, degener)"


In [19]:
import pyLDAvis.gensim_models

In [20]:
#Visu interractive affichant les topics, les mots etc
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.126726 -0.073635       1        1  23.943045
3      0.151105 -0.058778       2        1  16.605209
0     -0.163714  0.004902       3        1  12.007916
9      0.133016  0.232031       4        1   9.787500
2     -0.039891 -0.060084       5        1   8.343309
4     -0.016762 -0.016495       6        1   7.011542
1      0.119430 -0.063910       7        1   5.853656
6     -0.164075  0.013440       8        1   5.657264
7     -0.033013 -0.026835       9        1   5.606297
5     -0.112822  0.049364      10        1   5.184261, topic_info=         Term        Freq       Total Category  logprob  loglift
2412     fast  783.000000  783.000000  Default  30.0000  30.0000
34      sport  394.000000  394.000000  Default  29.0000  29.0000
836      wast  375.000000  375.000000  Default  28.0000  28.0000
52     health  292.000000  292.000000  Default  27.0000  27.0000
6        port  127.000000  127.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
370      come    3.464616   11.330656  Topic10  -5.7417   1.7746
433      like    3.854053   30.699807  Topic10  -5.6351   0.8844
548   healthi    3.873791   38.356976  Topic10  -5.6300   0.6668
479    diabet    3.736605  125.080098  Topic10  -5.6661  -0.5512
151      sure    3.183923    8.182141  Topic10  -5.8262   2.0157

[610 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
3663      7  0.622263        abnorm
3228      7  0.836265       accumul
993      10  0.841800          acid
1070      2  0.929887         advic
636       8  0.627930      advocaci
...     ...       ...           ...
1010      1  0.176845        youtub
1010      5  0.176845        youtub
1010     10  0.530536        youtub
1162      9  0.833840        zodiac
525       6  0.920821  ساعه_استجابه

[1085 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 4, 1, 10, 3, 5, 2, 7, 8, 6])

In [ ]:
# Utilisez la colonne "Texte corrigé" pour le prétraitement
processed_docs = [preprocess(doc) for doc in tweets_df['Texte corrigé']]

# Obtenez la distribution des sujets pour chaque tweet
tweet_topic_distributions = [lda_model.get_document_topics(dictionary.doc2bow(doc)) for doc in processed_docs]


In [ ]:
# Choisissez le sujet dominant pour chaque tweet
dominant_topics = [max(topic_dist, key=lambda x: x[1])[0] for topic_dist in tweet_topic_distributions]

# Ajoutez la colonne 'Dominant Topic' au DataFrame
tweets_df['Dominant Topic'] = dominant_topics


In [ ]:
# Créer une Mapping pour les Topics
topic_mapping = {}
for i in range(lda_model.num_topics):
    topic_mapping[i] = 'Topic {}'.format(i+1)


In [ ]:
tweets_df['Topic Label'] = [topic_mapping[dominant_topic] for dominant_topic in dominant_topics]


In [ ]:
# Affichez les premières lignes du DataFrame
tweets_df.head()
